In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import yfinance as yf
import pandas as pd
from pandas_datareader import wb

# Define functions
def download_sector_indices(indices, start_date, end_date):
    """
    Download historical data for Nifty sector-based indices.
    """
    data = {}
    for index in indices:
        data[index] = yf.download(index, start=start_date, end=end_date)
    sector_data = {index: data[index]['Close'] for index in data}
    df_sector_data = pd.DataFrame(sector_data)
    return df_sector_data

def download_macro_data(indicators, country, start_year, end_year):
    """
    Download macroeconomic data from World Bank.
    """
    macro_data = wb.download(indicator=list(indicators.keys()), country=country, start=start_year, end=end_year)
    macro_data = macro_data.rename(columns=indicators)
    macro_data = macro_data.reset_index()
    return macro_data

def merge_data(sector_data, macro_data):
    """
    Merge sector indices data with macroeconomic data.
    """
    macro_data['year'] = pd.to_datetime(macro_data['year'])
    macro_data.set_index('year', inplace=True)
    macro_data = macro_data.resample('A').last()
    combined_data = sector_data.join(macro_data, how='outer')
    return combined_data

# Example usage
indices = ['^CNXAUTO', '^CNXBANK', '^CNXIT', '^CNXPHARMA', '^CNXREALTY']
start_date = '2014-01-01'
end_date = '2024-01-01'
df_sector_data = download_sector_indices(indices, start_date, end_date)

indicators = {'NY.GDP.MKTP.CD': 'GDP', 'FP.CPI.TOTL': 'CPI'}
country = 'IN'
start_year = 2014
end_year = 2023
df_macro_data = download_macro_data(indicators, country, start_year, end_year)

combined_data = merge_data(df_sector_data, df_macro_data)
print(combined_data.head())


In [ ]:
from pandas_datareader import wb

# List of macroeconomic indicators
indicators = {'NY.GDP.MKTP.CD': 'GDP', 'FP.CPI.TOTL': 'CPI'}

# Download data
macro_data = wb.download(indicator=list(indicators.keys()), country='IN', start=2014, end=2023)
macro_data = macro_data.rename(columns=indicators)
print(macro_data.head())


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Example for one sector index
sector_index = df_sector_data['^CNXBANK']
train_data = sector_index[:'2021']
test_data = sector_index['2022':]

# Fit ARIMA model
model = ARIMA(train_data, order=(5, 1, 0))
model_fit = model.fit()
predictions = model_fit.forecast(steps=len(test_data))

# Compare predictions with actual values
print(predictions)
